## 1. Khởi tạo Spark Session

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, current_timestamp
import os

# Set AWS environment variables for MinIO
os.environ['AWS_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admin123'

# Khởi tạo Spark Session
spark = (
    SparkSession.builder.appName("Reset_EntityNameNormalized")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "1536m")
    .config("spark.executor.cores", "2")
    # ===== Iceberg Catalog qua Nessie =====
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://gold/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    # ===== Cấu hình MinIO =====
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    # ===== Spark + Hadoop S3 connector =====
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print("✓ Spark Session đã được khởi tạo!")

## 2. Kiểm tra dữ liệu hiện tại

## 1.5. Làm sạch bộ nhớ cache

In [ ]:
# Làm sạch cache và bộ nhớ
spark.catalog.clearCache()
print("✓ Đã xóa cache!")

# Unpersist tất cả các DataFrame đang cached
for table in spark.catalog.listTables():
    try:
        spark.catalog.uncacheTable(f"{table.database}.{table.name}")
    except:
        pass

print("✓ Đã unpersist các bảng cached!")
print("✓ Bộ nhớ đã được làm sạch, sẵn sàng chạy reset!")

In [ ]:
# Đọc bảng Post_Entity
df = spark.sql("SELECT * FROM nessie.gold_result_model_multi_task.Post_Entity LIMIT 10")

print("Dữ liệu hiện tại:")
df.select("postID", "entityID", "entityOrder", "entityName", "entityNameNormalized").show(10, truncate=False)

# Thống kê
total_records = spark.sql("SELECT COUNT(*) as total FROM nessie.gold_result_model_multi_task.Post_Entity").collect()[0]['total']
normalized_records = spark.sql("SELECT COUNT(*) as total FROM nessie.gold_result_model_multi_task.Post_Entity WHERE entityNameNormalized IS NOT NULL").collect()[0]['total']

print(f"\nThống kê:")
print(f"   • Tổng số records: {total_records:,}")
print(f"   • Records có entityNameNormalized: {normalized_records:,}")
print(f"   • Records NULL: {total_records - normalized_records:,}")

## 3. Reset cột entityNameNormalized

**Chọn 1 trong 3 option:**
- **Option 1:** Reset TOÀN BỘ (set NULL cho tất cả)
- **Option 2:** Reset chỉ entityType = 'MAJ'
- **Option 3:** Reset chỉ entityType = 'ORG'

### Option 1: Reset TOÀN BỘ bảng Post_Entity

In [ ]:
#  Thao tác này sẽ xóa TOÀN BỘ dữ liệu entityNameNormalized!

print("Đang reset TOÀN BỘ cột entityNameNormalized...")

# Làm sạch cache trước khi chạy
spark.catalog.clearCache()

# Đọc toàn bộ dữ liệu
df_all = spark.sql("""
    SELECT 
        postID,
        entityID,
        entityOrder,
        entityName,
        created_at
    FROM nessie.gold_result_model_multi_task.Post_Entity
""")

# Thêm cột entityNameNormalized = NULL và updated_at
df_reset = df_all.withColumn("entityNameNormalized", lit(None).cast("string")) \
                 .withColumn("updated_at", current_timestamp())

# Tạo temporary view
df_reset.createOrReplaceTempView("temp_reset_all")

# MERGE để update
spark.sql("""
    MERGE INTO nessie.gold_result_model_multi_task.Post_Entity AS target
    USING temp_reset_all AS source
    ON target.postID = source.postID 
        AND target.entityID = source.entityID
        AND target.entityOrder = source.entityOrder
    WHEN MATCHED THEN UPDATE SET
        target.entityNameNormalized = source.entityNameNormalized,
        target.updated_at = source.updated_at
""")

# Xóa temporary view và DataFrame
spark.catalog.dropTempView("temp_reset_all")
df_reset.unpersist()
df_all.unpersist()

print("✓ Đã reset toàn bộ cột entityNameNormalized = NULL!")

# Xác minh
result = spark.sql("""
    SELECT COUNT(*) as total_null 
    FROM nessie.gold_result_model_multi_task.Post_Entity 
    WHERE entityNameNormalized IS NULL
""").collect()[0]['total_null']

print(f"✓ Số records có entityNameNormalized = NULL: {result:,}")

### Option 2: Reset chỉ entityType = 'MAJ'

In [ ]:
# Reset chỉ các records có entityType = 'MAJ'

print("Đang reset entityNameNormalized cho entityType = 'MAJ'...")

# Làm sạch cache trước khi chạy
spark.catalog.clearCache()

# Đọc dữ liệu với JOIN Entity để lọc MAJ
df_maj = spark.sql("""
    SELECT 
        pe.postID,
        pe.entityID,
        pe.entityOrder,
        pe.entityName,
        pe.created_at
    FROM nessie.gold_result_model_multi_task.Post_Entity pe
    INNER JOIN nessie.gold_result_model_multi_task.Entity e
        ON pe.entityID = e.entityID
    WHERE e.entityType = 'MAJ'
""")

# Set entityNameNormalized = NULL
df_reset_maj = df_maj.withColumn("entityNameNormalized", lit(None).cast("string")) \
                     .withColumn("updated_at", current_timestamp())

# Tạo temporary view
df_reset_maj.createOrReplaceTempView("temp_reset_maj")

# MERGE để update chỉ MAJ
spark.sql("""
    MERGE INTO nessie.gold_result_model_multi_task.Post_Entity AS target
    USING temp_reset_maj AS source
    ON target.postID = source.postID 
        AND target.entityID = source.entityID
        AND target.entityOrder = source.entityOrder
    WHEN MATCHED THEN UPDATE SET
        target.entityNameNormalized = source.entityNameNormalized,
        target.updated_at = source.updated_at
""")

# Xóa temporary view và DataFrame
spark.catalog.dropTempView("temp_reset_maj")
df_reset_maj.unpersist()
df_maj.unpersist()

print("✓ Đã reset entityNameNormalized cho entityType = 'MAJ'!")

# Xác minh
result = spark.sql("""
    SELECT COUNT(*) as total_null 
    FROM nessie.gold_result_model_multi_task.Post_Entity pe
    INNER JOIN nessie.gold_result_model_multi_task.Entity e 
        ON pe.entityID = e.entityID
    WHERE e.entityType = 'MAJ' 
    AND pe.entityNameNormalized IS NULL
""").collect()[0]['total_null']

print(f"✓ Số records MAJ có entityNameNormalized = NULL: {result:,}")

### Option 3: Reset chỉ entityType = 'ORG'

In [ ]:
# Reset chỉ các records có entityType = 'ORG'

print("Đang reset entityNameNormalized cho entityType = 'ORG'...")

# Làm sạch cache trước khi chạy
spark.catalog.clearCache()

# Đọc dữ liệu với JOIN Entity để lọc ORG
df_org = spark.sql("""
    SELECT 
        pe.postID,
        pe.entityID,
        pe.entityOrder,
        pe.entityName,
        pe.created_at
    FROM nessie.gold_result_model_multi_task.Post_Entity pe
    INNER JOIN nessie.gold_result_model_multi_task.Entity e
        ON pe.entityID = e.entityID
    WHERE e.entityType = 'ORG'
""")

# Set entityNameNormalized = NULL
df_reset_org = df_org.withColumn("entityNameNormalized", lit(None).cast("string")) \
                     .withColumn("updated_at", current_timestamp())

# Tạo temporary view
df_reset_org.createOrReplaceTempView("temp_reset_org")

# MERGE để update chỉ ORG
spark.sql("""
    MERGE INTO nessie.gold_result_model_multi_task.Post_Entity AS target
    USING temp_reset_org AS source
    ON target.postID = source.postID 
        AND target.entityID = source.entityID
        AND target.entityOrder = source.entityOrder
    WHEN MATCHED THEN UPDATE SET
        target.entityNameNormalized = source.entityNameNormalized,
        target.updated_at = source.updated_at
""")

# Xóa temporary view và DataFrame
spark.catalog.dropTempView("temp_reset_org")
df_reset_org.unpersist()
df_org.unpersist()

print("✓ Đã reset entityNameNormalized cho entityType = 'ORG'!")

# Xác minh
result = spark.sql("""
    SELECT COUNT(*) as total_null 
    FROM nessie.gold_result_model_multi_task.Post_Entity pe
    INNER JOIN nessie.gold_result_model_multi_task.Entity e 
        ON pe.entityID = e.entityID
    WHERE e.entityType = 'ORG' 
    AND pe.entityNameNormalized IS NULL
""").collect()[0]['total_null']

print(f"✓ Số records ORG có entityNameNormalized = NULL: {result:,}")

## 4. Kiểm tra kết quả sau khi reset

In [ ]:
# Xem lại dữ liệu sau khi reset
df_after = spark.sql("""
    SELECT 
        pe.postID,
        pe.entityID,
        pe.entityOrder,
        pe.entityName,
        pe.entityNameNormalized,
        e.entityType
    FROM nessie.gold_result_model_multi_task.Post_Entity pe
    INNER JOIN nessie.gold_result_model_multi_task.Entity e 
        ON pe.entityID = e.entityID
    LIMIT 20
""")

print("\nDữ liệu sau khi reset:")
df_after.show(20, truncate=False)

# Thống kê theo entityType
stats = spark.sql("""
    SELECT 
        e.entityType,
        COUNT(*) as total_records,
        SUM(CASE WHEN pe.entityNameNormalized IS NULL THEN 1 ELSE 0 END) as null_count,
        SUM(CASE WHEN pe.entityNameNormalized IS NOT NULL THEN 1 ELSE 0 END) as not_null_count
    FROM nessie.gold_result_model_multi_task.Post_Entity pe
    INNER JOIN nessie.gold_result_model_multi_task.Entity e 
        ON pe.entityID = e.entityID
    GROUP BY e.entityType
    ORDER BY e.entityType
""")

print("\nThống kê theo entityType:")
stats.show(truncate=False)

## 5. Dừng Spark Session

In [ ]:
# Dừng Spark Session
spark.stop()
print("✓ Spark Session đã được dừng!")